# **Launch Sites Locations Analysis with Folium**


In this module of the project, we make use of `Folium` for iteractive visualization.

## Objectives


- Mark all launch sites on a map
- Mark the success/failed launches for each site on the map
- Calculate the distances between a launch site to its proximities

### Importing required Python packages

In [7]:
# !pip3 install folium
# !pip3 install wget

In [1]:
import folium
import wget
import pandas as pd
import gc, os

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

#### Marking all launch sites on a map

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [3]:
file = "/datasets/spacex_launch_geo.csv"
local_path = os.getcwd() + file
try:
    spacex_df=pd.read_csv(local_path)
    print('File found in local system.')
except(FileNotFoundError):
    print('File not found in local system. Downloading.')
    spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
    spacex_csv_file = wget.download(local_path)


File found in local system.


In [4]:
del file, local_path
gc.collect()

31

Now, let's take a look at what are the coordinates for each site.

In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame `launch_sites`


In [9]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for site, lat, long in zip(launch_sites_df['Launch Site'], launch_sites_df.Lat, launch_sites_df.Long):
    circle = folium.Circle([lat, long],
                           radius=1000, 
                           color='#d35400',
                           fill=True).add_child(folium.Popup(site))
    
    marker = folium.map.Marker([lat, long],
                               icon=DivIcon(icon_size=(20,20),
                                            icon_anchor=(0,0),
                                            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

#### Marking the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not.

Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()

Now, let's create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value

In [11]:
spacex_df['marker_color'] = spacex_df['class'].replace({0:'red',1:'blue'})

For each launch result in `spacex_df` data frame, let us add a `folium.Marker` to `marker_cluster`

In [12]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    marker = folium.map.Marker([record[1], record[2]],
                               icon=folium.Icon(color='white', icon_color=record[4]),
                               popup=record[0]
    )
    marker_cluster.add_child(marker)

site_map

/tmp/ipykernel_160282/784894097.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  marker = folium.map.Marker([record[1], record[2]],
/tmp/ipykernel_160282/784894097.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  icon=folium.Icon(color='white', icon_color=record[4]),
/tmp/ipykernel_160282/784894097.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  popup=record[0]


From the color-labeled markers in marker clusters, it should be able to easily identify which launch sites have relatively high success rates.


#### Calculating the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Calculating the distance between two points on the map based on their `Lat` and `Long` values using the following function:

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [15]:
coordinates_launchsite = [launch_sites_df.loc[1,'Lat'], launch_sites_df.loc[1,'Long']]
coordinates_launchsite

[28.56319718, -80.57682003]

In [16]:
coordinates_coastline = [28.56367, -80.56815] # using moise pointer 
distance_coastline = calculate_distance(*coordinates_launchsite, *coordinates_coastline)
distance_coastline

0.8486225666350655

In [17]:
coordinates_airport = [28.342287, -80.68927]
distance_airport = calculate_distance(*coordinates_launchsite, *coordinates_airport)
distance_airport

26.920378075027987

In [18]:
coordinates_orlando = [28.54531, -81.39084]
distance_orlando = calculate_distance(*coordinates_launchsite, *coordinates_orlando)
distance_orlando

79.5547903546591

In [19]:
coordinates_railroad = [28.56344, -80.58691]
distance_railroad = calculate_distance(*coordinates_launchsite, *coordinates_railroad)
distance_railroad

0.9860789816287603

After obtaining its coordinate, let us create a `folium.Marker` to show the distance:

In [20]:
distance_marker_coast = folium.Marker(
   coordinates_coastline,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 40; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
       )
   )
distance_marker_coast.add_to(site_map)

In [21]:
distance_marker_airport = folium.Marker(
   coordinates_airport,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 40; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_airport),
       )
   )
distance_marker_airport.add_to(site_map)

In [22]:
distance_marker_orlando = folium.Marker(
   coordinates_orlando,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 40; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_orlando),
       )
   )
distance_marker_orlando.add_to(site_map)

In [23]:
distance_marker_railroad = folium.Marker(
   coordinates_railroad,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 40; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railroad),
       )
   )
distance_marker_railroad.add_to(site_map)

Drawing a `PolyLine` between a launch site to the selected coastline point

In [24]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[coordinates_launchsite,coordinates_coastline], weight=1)
site_map.add_child(lines)

line_airport=folium.PolyLine(locations=[coordinates_launchsite,coordinates_airport], weight=1)
site_map.add_child(line_airport)

line_orlando=folium.PolyLine(locations=[coordinates_launchsite,coordinates_orlando], weight=2)
site_map.add_child(line_orlando)

line_railroad=folium.PolyLine(locations=[coordinates_launchsite,coordinates_railroad], weight=1)
site_map.add_child(line_railroad)

Copyright © 2021 IBM Corporation. All rights reserved.
